## Installing Package dan Importing Module

In [ ]:
# Installing this package to connected our python file with our dataset that we already have in mysql
!pip install mysql-connector-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 66.3 MB/s eta 0:00:00


In [ ]:
# Importing mysql.connector module that's been installed
import mysql.connector

In [ ]:
# Importing tensorflow dan tensorflow_hub(for calling pre-trained embedding tensorflow model) modules.
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
# Importing pandas module that used for processing dataframe (in here is our dataset which is in csv format)
import pandas as pd

In [ ]:
# Importing math module that used for called pi (22/7) values (to scoring our cosine similiraties)
import math

In [ ]:
# Importing regex module that used for pre-processing (cleaning text data)
import re

In [ ]:
# Importing nltk module that used for pre-processing (cleaning text data)
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

# Calling Pretrained Encoder Tensorflow Model 

This is a pre-trained encoder Tensorflow model that Tensorflow suggested to make
*text similarty* model. 

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


# Dataset

In [ ]:
# Replacing the placeholders with your own values
connection_name = 'c23df02-diskusai:asia-southeast2:diskusaidataset'
ip_address = '34.101.70.233'
db_name = 'diskusaidataset'
user = 'root'
password = 'df02'

# Creating the connection string
connection_string = f"mysql+mysqlconnector://{user}:{password}@{ip_address}/{db_name}?unix_socket=/cloudsql/{connection_name}"

# Establishing the connection
cnx = mysql.connector.connect(user=user, password=password, host=ip_address, database=db_name)

In [ ]:
# Replacing 'your_table_name' with the actual name of your table
query = "SELECT * FROM diskusaidataset"

# Executing the query
cursor = cnx.cursor()
cursor.execute(query)

# Fetching all the results
data = cursor.fetchall()

# Converting the data into a Pandas DataFrame
dataset = pd.DataFrame(data, columns=cursor.column_names)

# Closeing the cursor and connection
cursor.close()
cnx.close()

In [ ]:
# Cleaning question column from special characters that don't exist in regex module
dataset['question'] = dataset['question'].str.replace('<p>', '')
dataset['question'] = dataset['question'].str.replace('<h2>', '')
dataset['question'] = dataset['question'].str.replace('</h2>', '')
dataset['question'] = dataset['question'].str.replace('<h3>', '')
dataset['question'] = dataset['question'].str.replace('</h3>', '')
dataset['question'] = dataset['question'].str.replace('</p>', '')
dataset['question'] = dataset['question'].str.replace('<strong>', '')
dataset['question'] = dataset['question'].str.replace('</strong>', '')

In [ ]:
# Cleaning image link data from dataset 
dataset = dataset[dataset['question'].str.contains('<img src=')==False]

In [ ]:
# Re-indexing dataset after cleaning process
dataset=dataset.reset_index(drop=True)

In [ ]:
# Creating new column that contained questions links
link_diskusi=[]
list_course_id=(dataset['course_id'].values).tolist()
list_id=(dataset['id'].values).tolist()
for i in range(len(dataset)):
  link_diskusi.append('https://www.dicoding.com/academies/'+str(list_course_id[i])+'/discussions/'+str(list_id[i]))
dataset['link_diskusi']=link_diskusi

In [ ]:
# Cleaning Data Text

STOPWORDS1 = set(stopwords.words('indonesian'))
STOPWORDS2 = set(stopwords.words('english'))
MIN_WORDS = 4
MAX_WORDS = 200

PATTERN_S = re.compile("\'s")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 

# Defining function to clean data text using regex module
def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers.
        text (str): input text
    return (str): modified initial text
    """
    text = text.lower()  # lowercase text
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

# Defining tokenizer1 to clean data text using indonesian stopwords
def tokenizer1(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS1, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    stemmer = WordNetLemmatizer()
    tokens1 = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    tokens1 = ' '.join(tokens1)
    return tokens1    

# Defining tokenizer2 to clean data text using english stopwords
def tokenizer2(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS2, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    stemmer = WordNetLemmatizer()
    tokens2 = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    tokens2 = ' '.join(tokens2)
    return tokens2    

# Defining clean_sentences to apply all cleaning data text process to our dataset
def clean_sentences(df):
    """
    Remove irrelavant characters (in new column clean_sentence).
    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).
    """
    print('Cleaning sentences...')
    df['question'] = df['question'].apply(clean_text)
    df['question'] = df['question'].apply(
        lambda x: tokenizer1(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS1, lemmatize=True))
    df['question'] = df['question'].apply(
        lambda x: tokenizer2(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS2, lemmatize=True))
    return df

# Applying all cleaning data text process to our dataset
dataset = clean_sentences(dataset)

# Creating tokes from our cleaned dataset
tokens = dataset['question'].str.split()
tokens = [word for sublist in tokens for word in sublist]

Cleaning sentences...


# Defining Function That Named "chatbot" to Check *Cosine Similarities* Values Scores Between Our Input (diskusAI Chat-Bot's Question Input) Relatives to Questions that Already Exist in Our Dataset.

*   This function has 2 parameters that need to be filled to become local variables, our dataset and diskusAI chat-bot's question input.
*   That question input, which is in a string format, will be transformed to dataframe format.
*   After that, this function will embed every element in question input dataframe (in here is dataframe called data) and question column from main dataset (that given by Dicoding Indonesia) using pre-trained encoder Tensorflow model.
*   When that all elements have already embedded, this function will calculate cosine similarities values beetween question input relatives to questions that already exist in dataset, and then scoring that values.
*   Also making list of that score, and then add that as a new column from our main dataset.
*   After that, this function creates new list that contained top 10 scores of cosine similarities values then makes all of list' elements as index for calling top 10 modules name that the most related with question input also top 10 discussion titles and those links.
*  This function returns list a (contained top 10 modules name related with question input), b (contained top 10 discussion titles related with question input), and c (contained top 10 discussion titles' links related with question input).








In [ ]:
def chatbot(data_set,pertanyaan):
  xx=[]
  xx.append(pertanyaan)
  data = {'input': xx}  
  data_input = pd.DataFrame(data)
  cos_scores=[]
  for i in range(len(data_set)):
    sts_encode1 = tf.nn.l2_normalize(embed(tf.constant(data_input['input'].tolist())), axis=1)
    sts_encode2 = tf.nn.l2_normalize(embed(tf.constant(dataset['question'][i:i+1])), axis=1)
    cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
    clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
    scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
    cos_scores.append(scores)

  cos_scores_list=[]
  for i in range(len(cos_scores)):
    cos_scores_list_tensor=cos_scores[i-1][0].numpy()
    cos_scores_list.append(float(cos_scores_list_tensor))

  dataset['Cosines Similarity Scores']=cos_scores_list

  max_cos_scores_list=[]
  a_list=[]
  b_list=[]
  c_list=[]

  for i in range(10):
    max_cos=max(cos_scores_list)
    max_cos_scores_list.append(max_cos)
    cos_scores_list.remove(max_cos)
    dataset_cos_max = dataset.loc[dataset['Cosines Similarity Scores'] == max_cos_scores_list[i]]
    a_list.append(dataset_cos_max['module_name'].values.tolist())
    b_list.append(dataset_cos_max['discussion_title'].values.tolist())
    c_list.append(dataset_cos_max['link_diskusi'].values.tolist())
  a=[]
  b=[]
  c=[]
  for i in range(10):
    a.append(a_list[i][0])
    b.append(b_list[i][0])
    c.append(c_list[i][0])
  return a, b, c

# Simulation

In [ ]:
print('Selamat Datang di ChatBot DiskusAI!')
print('-----------------------------------')
print('-----------------------------------')

while True:
    text=str(input('Masukkan Pertanyaan Kamu (Ketik STOP Jika Ingin Mengakhiri Sesi) :'))
    if text=='STOP':
      print('Sampai Jumpa, Terima Kasih Telah Menggunakan DiskusAI! <3')
      break
    else:
      a, b, c=chatbot(dataset,text)
      print('-----------------------------------')
      print('Berikut Modul yang Relevan dengan Pertanyaan Kamu: ')
      print('-----------------------------------')
      for j in range(len(a)):
        print(j+1,'.',a[j])
      print('-----------------------------------')
      print('-----------------------------------')
      print('Berikut Judul Diskusi yang Relevan dengan Pertanyaan Kamu: ')
      print('-----------------------------------')
      for k in range(len(b)):
        print(k+1,'.',b[k])
      print('-----------------------------------')
      print('-----------------------------------')
      print('Berikut Link Judul Diskusi yang Relevan dengan Pertanyaan Kamu: ')
      print('-----------------------------------')
      for l in range(len(c)):
        print(l+1,'.',c[l])
      print('-----------------------------------')

Selamat Datang di ChatBot DiskusAI!
-----------------------------------
-----------------------------------
Masukkan Pertanyaan Kamu (Ketik STOP Jika Ingin Mengakhiri Sesi) :java script python?
-----------------------------------
Berikut Modul yang Relevan dengan Pertanyaan Kamu: 
-----------------------------------
1 . Kuis Coding : Function
2 . Submission: Tugas Akhir Membuat Website
3 . Pendahuluan Web Components
4 . Mekanisme Belajar
5 . If/Else Statement
6 . Menghubungkan Berkas CSS dengan HTML
7 . Shadow DOM
8 . Mekanisme Belajar
9 . Kuis Koding: Asynchronous Proses secara Berantai
10 . Submission: Membuat Aplikasi Web dengan ES6, Custom Element, NPM, Module Bundler, dan AJAX
-----------------------------------
-----------------------------------
Berikut Judul Diskusi yang Relevan dengan Pertanyaan Kamu: 
-----------------------------------
1 . fungsi minimal belom berjalan dengan baik. maaf kak ini salah dimana ya?
2 . Apakah bila saya tidak meng ikuti webinar sertifikat yang di